[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mohamed-Mehira/AI-computer-vision/blob/master/Tensorflow/examples/lego_model.ipynb)

In [ ]:
import os
import math
import random
import shutil
import math
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# tf.keras.backend.clear_session()
from importlib import reload
reload(keras.models)

In [ ]:
# reading class names

BASE_dir = '../Datasets/lego/'

class_names_df = pd.read_csv(BASE_dir + 'metadata.csv', encoding='ISO-8859-1')
class_names = list(class_names_df['minifigure_name'])

tf.random.set_seed(1)

In [ ]:
# importing our data

def change_path(path):
    path = BASE_dir + path
    return path

dataset_df = pd.read_csv(BASE_dir + 'index.csv', encoding='ISO-8859-1')
test_data = pd.read_csv(BASE_dir + 'test.csv', encoding='ISO-8859-1')

dataset_df.path = dataset_df.path.apply(change_path)
test_data.path = test_data.path.apply(change_path)

print(dataset_df.shape)
test_data

In [ ]:
# splitting our data into training and validation

train_data = pd.DataFrame()
val_data = pd.DataFrame()
val_percentage = 0.3

grouped_df = dataset_df.groupby('class_id')
for group_name, group_data in grouped_df:
    val_samples = group_data.sample(frac=val_percentage, random_state=42)
    val_data = pd.concat([val_data, val_samples])
    
    train_samples = group_data.drop(val_samples.index)
    train_data = pd.concat([train_data, train_samples])

train_data = train_data.sample(frac=1, random_state=42)
val_data = val_data.sample(frac=1, random_state=42)

train_data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)

print(train_data.shape)
print(val_data.shape)
train_data

In [ ]:
# splitting our data into images and labels

train_imgs = []
val_imgs = []
test_imgs = []

train_img_paths = train_data.path
val_img_paths = val_data.path
test_img_paths = test_data.path

for train_img_path in train_img_paths:
    train_img = mpimg.imread(train_img_path)
    train_img = cv2.resize(train_img, (256, 256))
    train_imgs.append(train_img)
    
for val_img_path in val_img_paths:
    val_img = mpimg.imread(val_img_path)
    val_img = cv2.resize(val_img, (256, 256))
    val_imgs.append(val_img)
    
for test_img_path in test_img_paths:
    test_img = mpimg.imread(test_img_path)
    test_img = cv2.resize(test_img, (256, 256))
    test_imgs.append(test_img)

train_imgs = np.array(train_imgs) / 255
val_imgs = np.array(val_imgs) / 255
test_imgs = np.array(test_imgs) / 255

train_labels = train_data.class_id.values
val_labels = val_data.class_id.values
test_labels = test_data.class_id.values

print(train_imgs.shape)
print(val_imgs.shape)
print(test_imgs.shape)

In [ ]:
def show(images, labels, n=9, pred_labels=None):
    plt.figure(figsize=(10,10))
    for i in range(n):
        x = int(math.sqrt(n))
        plt.subplot(x,x,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        image = images[i]
        plt.imshow(image, cmap=plt.cm.binary)
        label = class_names[labels[i]-1]
        if pred_labels is not None:
            label += "/ Pred:" + class_names[pred_labels[i]-1]
        plt.xlabel(label)
    plt.show()
    
show(train_imgs[20:60], train_labels[20:60], n=36)

In [ ]:
# using transfer learning

vgg_model = tf.keras.applications.vgg16.VGG16()
print(type(vgg_model))
# vgg_model.summary()

In [ ]:
model = keras.models.Sequential()
for layer in vgg_model.layers[0:-1]:
    layer.trainable = False
    model.add(layer)
    
model.summary()

model.add(layers.Dense(38))

In [ ]:
# using my own model

# model = keras.models.Sequential()
# model.add(layers.Conv2D(32, (3,3), strides=(1,1), padding="valid", activation='relu', input_shape=(256, 256, 3)))
# model.add(layers.MaxPool2D((2,2)))
# model.add(layers.Conv2D(64, 3, activation='relu'))
# model.add(layers.MaxPool2D((2,2)))
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(38))
# print(model.summary())

In [ ]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ['accuracy']

model.compile(optimizer=optim, loss=loss, metrics=metrics)

In [ ]:
# callbacks
# early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, verbose=2)

history = model.fit(train_imgs, train_labels, validation_data=(val_imgs, val_labels), batch_size=12, epochs=30, verbose=2)

In [ ]:
model.save('lego-classifier_model.h5')

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='valid loss')
plt.grid()
plt.legend(fontsize=15)

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='validation accuracy')
plt.grid()
plt.legend(fontsize=15);

In [ ]:
model.evaluate(test_batches, verbose=2)

In [ ]:
predictions = model.predict()